# Introducción a los Embeddings de Texto con Modelos Abiertos

Este cuaderno te enseñará los conceptos fundamentales de los **embeddings de texto** usando modelos abiertos disponibles en Hugging Face. Los embeddings son representaciones numéricas de palabras o frases que capturan su significado semántico de manera que las computadoras puedan procesarlo.

## ¿Qué vas a aprender?

1. **Qué son los embeddings** y por qué son útiles en el procesamiento de lenguaje natural
2. **Cómo generar embeddings** usando modelos pre-entrenados abiertos
3. **Calcular similitudes** entre textos usando embeddings
4. **Diferencias** entre embeddings de palabras y de oraciones
5. **Aplicaciones prácticas** de los embeddings
6. **Comparar diferentes modelos** para encontrar el mejor para español

## Ventajas de usar modelos abiertos

- **Gratuitos**: No hay costos por uso como con APIs comerciales
- **Transparentes**: Podés inspeccionar y entender cómo funcionan
- **Personalizables**: Los podés afinar para tu dominio específico
- **Privacidad**: Tus datos no salen de tu computadora

## Configuración del Entorno

Primero instalamos las librerías necesarias. Vamos a usar:
- **sentence-transformers**: Para generar embeddings de alta calidad
- **scikit-learn**: Para calcular similitudes
- **numpy**: Para manipulación de arrays
- **ipywidgets**: Para interfaces interactivas

Si estás corriendo esto localmente, descomentá la siguiente línea:

In [ ]:
!pip install sentence-transformers scikit-learn numpy ipywidgets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.4 MB/s eta 0:00:00


In [ ]:
# Importamos las librerías necesarias
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings
warnings.filterwarnings('ignore')  # Silenciamos warnings menores

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


## Selección del Modelo

Vamos a usar un modelo pre-entrenado que funciona bien con español. Te mostramos varias opciones:

1. **`paraphrase-multilingual-MiniLM-L12-v2`**: Rápido, funciona bien con español
2. **`distiluse-base-multilingual-cased`**: Buena calidad, multiidioma
3. **`all-MiniLM-L6-v2`**: Muy rápido, principalmente inglés pero funciona con español

Empezamos con el modelo multilingüe que da buenos resultados en español:

In [ ]:
# Cargamos el modelo de embeddings
modelo_nombre = 'sentence-transformers/distiluse-base-multilingual-cased-v2'
modelo_embeddings = SentenceTransformer(modelo_nombre)

print(f"✅ Modelo cargado: {modelo_nombre}")
print(f"📊 Dimensión de los embeddings: {modelo_embeddings.get_sentence_embedding_dimension()}")

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

✅ Modelo cargado: sentence-transformers/distiluse-base-multilingual-cased-v2
📊 Dimensión de los embeddings: 512


## Tu Primer Embedding

Vamos a generar el embedding de una palabra simple y ver qué aspecto tiene:

In [ ]:
# Generamos el embedding de una palabra
palabra = "vida"
embedding = modelo_embeddings.encode([palabra])

print(f"Palabra: '{palabra}'")
print(f"Longitud del embedding: {len(embedding[0])}")
print(f"Primeros 10 valores: {embedding[0][:10]}")
print(f"Tipo de datos: {type(embedding[0][0])}")

Palabra: 'vida'
Longitud del embedding: 512
Primeros 10 valores: [ 0.01135431  0.03165706  0.0005522  -0.00983733 -0.05544117 -0.02832641
 -0.04866723 -0.01849577 -0.02111436 -0.06133432]
Tipo de datos: <class 'numpy.float32'>


### ¿Qué acabamos de ver?

- **Un embedding es un vector**: Una lista de números que representa el significado de la palabra
- **Dimensionalidad fija**: Todos los embeddings tienen la misma longitud (384 en este modelo)
- **Números decimales**: Los valores pueden ser positivos o negativos, generalmente entre -1 y 1
- **Significado distribuido**: Cada dimensión captura algún aspecto del significado

## Embeddings de Oraciones

Los modelos modernos pueden generar embeddings que representan oraciones completas, considerando el contexto y el orden de las palabras:

In [ ]:
# Generamos el embedding de una oración completa
oracion = "¿Cuál es el sentido de la vida?"
embedding = modelo_embeddings.encode([oracion])

print(f"Oración: '{oracion}'")
print(f"Longitud del embedding: {len(embedding[0])}")
print(f"Primeros 10 valores: {embedding[0][:10]}")

Oración: '¿Cuál es el sentido de la vida?'
Longitud del embedding: 512
Primeros 10 valores: [ 0.01623319 -0.04003485  0.03519869  0.00145082 -0.07798815 -0.04161033
 -0.04850182  0.00988518 -0.00833548 -0.11256867]


## Calculando Similitudes

Una de las aplicaciones más útiles de los embeddings es medir qué tan similares son dos textos. Usamos la **similitud coseno**, que va de 0 (nada similar) a 1 (idéntico).

Probemos con tres oraciones de diferente similitud:

In [ ]:
# Definimos tres oraciones para comparar
oracion_1 = "¿Cuál es el sentido de la vida?"
oracion_2 = "¿Cómo podemos vivir una vida plena y significativa?"
oracion_3 = "¿Te gustaría una ensalada?"

# Generamos los embeddings
emb_1 = modelo_embeddings.encode([oracion_1])
emb_2 = modelo_embeddings.encode([oracion_2])
emb_3 = modelo_embeddings.encode([oracion_3])

print("🔍 Comparando similitudes:")
print(f"Oración 1: '{oracion_1}'")
print(f"Oración 2: '{oracion_2}'")
print(f"Oración 3: '{oracion_3}'")
print()

# Calculamos similitudes
sim_1_2 = cosine_similarity(emb_1, emb_2)[0][0]
sim_1_3 = cosine_similarity(emb_1, emb_3)[0][0]
sim_2_3 = cosine_similarity(emb_2, emb_3)[0][0]

print(f"📊 Similitud entre Oración 1 y 2: {sim_1_2:.3f}")
print(f"📊 Similitud entre Oración 1 y 3: {sim_1_3:.3f}")
print(f"📊 Similitud entre Oración 2 y 3: {sim_2_3:.3f}")

🔍 Comparando similitudes:
Oración 1: '¿Cuál es el sentido de la vida?'
Oración 2: '¿Cómo podemos vivir una vida plena y significativa?'
Oración 3: '¿Te gustaría una ensalada?'

📊 Similitud entre Oración 1 y 2: 0.613
📊 Similitud entre Oración 1 y 3: 0.171
📊 Similitud entre Oración 2 y 3: 0.077


### Interpretando los Resultados

- **Alta similitud (>0.7)**: Los textos tratan temas muy relacionados
- **Similitud media (0.3-0.7)**: Hay alguna relación semántica
- **Baja similitud (<0.3)**: Los textos son sobre temas diferentes

¿Los resultados coinciden con tu intuición? Las primeras dos oraciones deberían ser más similares entre sí que con la tercera.

## Embeddings de Palabras vs. Embeddings de Oraciones

Vamos a explorar la diferencia entre promediar embeddings de palabras individuales y generar embeddings de oraciones completas. Esto te ayudará a entender por qué los modelos modernos son mejores.

Usemos dos oraciones que tienen las mismas palabras pero significados diferentes:

In [ ]:
# Dos oraciones con las mismas palabras pero diferente significado
oracion_1 = "Los niños juegan en el parque"
oracion_2 = "La obra de teatro fue para niños en el parque"

print(f"Oración 1: '{oracion_1}'")
print(f"Oración 2: '{oracion_2}'")
print()

# Palabras clave (sin artículos, preposiciones, etc.)
palabras_clave = ["niños", "juegan", "parque", "obra", "teatro"]
print(f"Palabras clave extraídas: {palabras_clave}")

Oración 1: 'Los niños juegan en el parque'
Oración 2: 'La obra de teatro fue para niños en el parque'

Palabras clave extraídas: ['niños', 'juegan', 'parque', 'obra', 'teatro']


### Método 1: Promediando Embeddings de Palabras

Este es el método "naive" - simplemente promediamos los embeddings de cada palabra:

In [ ]:
# Generamos embeddings para cada palabra individualmente
embeddings_palabras = modelo_embeddings.encode(palabras_clave)
print(f"Forma del array de embeddings: {embeddings_palabras.shape}")
print(f"(Tenemos {len(palabras_clave)} palabras, cada una con {embeddings_palabras.shape[1]} dimensiones)")

# Calculamos el promedio
embedding_promedio = np.mean(embeddings_palabras, axis=0)
print(f"\nForma del embedding promedio: {embedding_promedio.shape}")
print(f"Primeros 5 valores: {embedding_promedio[:5]}")

### Método 2: Embeddings de Oraciones Completas

Ahora usamos el modelo para generar embeddings que consideran el contexto y el orden:

In [ ]:
# Generamos embeddings de las oraciones completas
emb_oracion_1 = modelo_embeddings.encode([oracion_1])
emb_oracion_2 = modelo_embeddings.encode([oracion_2])

print("Embeddings de oraciones completas:")
print(f"Oración 1 - Primeros 5 valores: {emb_oracion_1[0][:5]}")
print(f"Oración 2 - Primeros 5 valores: {emb_oracion_2[0][:5]}")

# Calculamos la similitud entre las dos oraciones
similitud_oraciones = cosine_similarity(emb_oracion_1, emb_oracion_2)[0][0]
print(f"\n📊 Similitud entre las oraciones: {similitud_oraciones:.3f}")

### Comparando los Métodos

Veamos qué tan diferentes son los embeddings generados por cada método:

In [ ]:
# Comparamos el embedding promedio con los embeddings de oraciones
# Nota: necesitamos ajustar las dimensiones para la comparación
embedding_promedio_reshaped = embedding_promedio.reshape(1, -1)

sim_promedio_oracion1 = cosine_similarity(embedding_promedio_reshaped, emb_oracion_1)[0][0]
sim_promedio_oracion2 = cosine_similarity(embedding_promedio_reshaped, emb_oracion_2)[0][0]

print("🔍 Resultados de la comparación:")
print(f"Similitud: Promedio de palabras vs Oración 1: {sim_promedio_oracion1:.3f}")
print(f"Similitud: Promedio de palabras vs Oración 2: {sim_promedio_oracion2:.3f}")
print(f"Similitud: Oración 1 vs Oración 2: {similitud_oraciones:.3f}")

print("\n💡 Observaciones:")
if abs(sim_promedio_oracion1 - sim_promedio_oracion2) < 0.1:
    print("- El método de promedio ve las oraciones como muy similares")
    print("- Esto se debe a que ignora el orden y el contexto")
else:
    print("- Incluso el promedio detecta algunas diferencias")

if similitud_oraciones < 0.8:
    print("- Los embeddings de oraciones capturan mejor las diferencias de significado")
    print("- Consideran el contexto y la estructura gramatical")

## Aplicaciones Prácticas

Los embeddings tienen muchas aplicaciones útiles. Veamos algunas:

### 1. Búsqueda Semántica

Podés buscar documentos por significado, no solo por palabras exactas:

In [ ]:
# Base de documentos de ejemplo
documentos = [
    "Cómo preparar una deliciosa pasta carbonara italiana",
    "Los beneficios del ejercicio físico para la salud mental",
    "Tutorial de programación en Python para principiantes",
    "Receta tradicional de empanadas argentinas",
    "Introducción al aprendizaje automático y redes neuronales",
    "Beneficios de la meditación y mindfulness para reducir estrés"
]

# Consulta del usuario
consulta = "quiero aprender a cocinar"

print(f"🔍 Buscando documentos relacionados con: '{consulta}'")
print()

# Generamos embeddings
emb_consulta = modelo_embeddings.encode([consulta])
emb_documentos = modelo_embeddings.encode(documentos)

# Calculamos similitudes
similitudes = cosine_similarity(emb_consulta, emb_documentos)[0]

# Ordenamos por similitud
indices_ordenados = np.argsort(similitudes)[::-1]

print("📊 Resultados ordenados por relevancia:")
for i, idx in enumerate(indices_ordenados):
    print(f"{i+1}. [{similitudes[idx]:.3f}] {documentos[idx]}")

🔍 Buscando documentos relacionados con: 'quiero aprender a cocinar'

📊 Resultados ordenados por relevancia:
1. [0.336] Cómo preparar una deliciosa pasta carbonara italiana
2. [0.272] Tutorial de programación en Python para principiantes
3. [0.169] Receta tradicional de empanadas argentinas
4. [0.165] Introducción al aprendizaje automático y redes neuronales
5. [0.041] Los beneficios del ejercicio físico para la salud mental
6. [0.033] Beneficios de la meditación y mindfulness para reducir estrés


### 2. Agrupamiento de Textos

Podés agrupar textos similares automáticamente:

In [ ]:
# Comentarios de usuarios sobre diferentes temas
comentarios = [
    "Me encanta la pizza margherita",
    "El fútbol argentino es apasionante",
    "Python es un lenguaje muy útil",
    "Las empanadas están deliciosas",
    "Messi es el mejor jugador del mundo",
    "JavaScript es complicado a veces",
    "Prefiero el asado a la parrilla",
    "La programación requiere práctica"
]

# Generamos embeddings
emb_comentarios = modelo_embeddings.encode(comentarios)

# Calculamos matriz de similitudes
matriz_similitudes = cosine_similarity(emb_comentarios)

print("🔍 Encontrando comentarios similares (similitud > 0.3):")
print()

for i in range(len(comentarios)):
    similares = []
    for j in range(len(comentarios)):
        if i != j and matriz_similitudes[i][j] > 0.3:
            similares.append((j, matriz_similitudes[i][j]))

    if similares:
        similares.sort(key=lambda x: x[1], reverse=True)
        print(f"'{comentarios[i]}'")
        for j, sim in similares:
            print(f"  └─ [{sim:.3f}] '{comentarios[j]}'")
        print()

🔍 Encontrando comentarios similares (similitud > 0.3):

'Me encanta la pizza margherita'
  └─ [0.545] 'Las empanadas están deliciosas'
  └─ [0.510] 'Prefiero el asado a la parrilla'

'El fútbol argentino es apasionante'
  └─ [0.545] 'Messi es el mejor jugador del mundo'
  └─ [0.342] 'Las empanadas están deliciosas'

'Python es un lenguaje muy útil'
  └─ [0.385] 'JavaScript es complicado a veces'

'Las empanadas están deliciosas'
  └─ [0.545] 'Me encanta la pizza margherita'
  └─ [0.419] 'Prefiero el asado a la parrilla'
  └─ [0.342] 'El fútbol argentino es apasionante'

'Messi es el mejor jugador del mundo'
  └─ [0.545] 'El fútbol argentino es apasionante'

'JavaScript es complicado a veces'
  └─ [0.385] 'Python es un lenguaje muy útil'

'Prefiero el asado a la parrilla'
  └─ [0.510] 'Me encanta la pizza margherita'
  └─ [0.419] 'Las empanadas están deliciosas'



## Experimentá con Tus Propios Textos

Ahora es tu turno. Probá con textos que te interesen:

In [ ]:
# Cambiá estos textos por los que quieras probar
mis_textos = [
    "Escribí aquí tu primer texto",
    "Y aquí tu segundo texto",
    "Podés agregar más textos si querés"
]

# Generamos embeddings y calculamos similitudes
mis_embeddings = modelo_embeddings.encode(mis_textos)
mis_similitudes = cosine_similarity(mis_embeddings)

print("🔍 Similitudes entre tus textos:")
for i in range(len(mis_textos)):
    for j in range(i+1, len(mis_textos)):
        sim = mis_similitudes[i][j]
        print(f"Texto {i+1} vs Texto {j+1}: {sim:.3f}")
        print(f"  '{mis_textos[i]}'")
        print(f"  '{mis_textos[j]}'")
        print()

## Comparación de Modelos: Probá los Mejores para Español

Ahora que entendés los conceptos básicos, es hora de comparar diferentes modelos para ver cuál funciona mejor para tus necesidades. Incluimos los nuevos **EmbeddingGemma** de Google y otros modelos top-rankeados para español.

In [ ]:
# Instalamos ipywidgets para el menú desplegable interactivo
# !pip install ipywidgets

import ipywidgets as widgets
from IPython.display import display, clear_output
import time
import gc

# Definimos los modelos a comparar
modelos_disponibles = {
    "Google EmbeddingGemma (Nuevo)": "google/embeddinggemma-300m",
    "Multilingual MiniLM (Rápido)": "paraphrase-multilingual-MiniLM-L12-v2",
    "E5 Multilingual Large (Alta Calidad)": "intfloat/multilingual-e5-large",
    "Sentence-BERT Español": "sentence-transformers/distiluse-base-multilingual-cased-v2",
    "Universal Sentence Encoder": "sentence-transformers/distiluse-base-multilingual-cased",
    "BGE Multilingual (Estado del Arte)": "BAAI/bge-m3"
}

print("🎯 Modelos disponibles para comparación:")
for nombre, modelo_id in modelos_disponibles.items():
    print(f"• {nombre}: {modelo_id}")

🎯 Modelos disponibles para comparación:
• Google EmbeddingGemma (Nuevo): google/embeddinggemma-300m
• Multilingual MiniLM (Rápido): paraphrase-multilingual-MiniLM-L12-v2
• E5 Multilingual Large (Alta Calidad): intfloat/multilingual-e5-large
• Sentence-BERT Español: sentence-transformers/distiluse-base-multilingual-cased-v2
• Universal Sentence Encoder: sentence-transformers/distiluse-base-multilingual-cased
• BGE Multilingual (Estado del Arte): BAAI/bge-m3


### Menú Interactivo de Comparación

Usá el menú desplegable para seleccionar un modelo y probar su rendimiento con oraciones en español:

In [ ]:
# Oraciones de prueba en español para evaluar los modelos
oraciones_prueba = [
    "Me gusta mucho la comida argentina",
    "Disfruto de la gastronomía de Argentina",
    "El fútbol es mi deporte favorito",
    "Prefiero mirar partidos de soccer",
    "Necesito ayuda con mi tarea de matemáticas",
    "¿Podés explicarme este problema de álgebra?",
    "El clima está muy lindo hoy",
    "Hace un día hermoso para salir",
    "Mañana tengo un examen importante",
    "Voy al supermercado a comprar verduras"
]

# Creamos el widget de selección
selector_modelo = widgets.Dropdown(
    options=list(modelos_disponibles.keys()),
    value=list(modelos_disponibles.keys())[0],
    description='Modelo:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

boton_evaluar = widgets.Button(
    description='🔍 Evaluar Modelo',
    button_style='primary',
    layout=widgets.Layout(width='200px')
)

area_resultados = widgets.Output()

# Variable global para almacenar el modelo actual
modelo_actual = None
nombre_modelo_actual = None

def cargar_modelo(nombre_modelo):
    """Carga un modelo de manera segura con manejo de errores"""
    global modelo_actual, nombre_modelo_actual

    modelo_id = modelos_disponibles[nombre_modelo]

    try:
        # Liberamos memoria del modelo anterior
        if modelo_actual is not None:
            del modelo_actual
            gc.collect()

        print(f"⏳ Cargando {nombre_modelo}...")
        start_time = time.time()

        # Intentamos cargar el modelo
        modelo_actual = SentenceTransformer(modelo_id)
        nombre_modelo_actual = nombre_modelo

        load_time = time.time() - start_time
        dimensiones = modelo_actual.get_sentence_embedding_dimension()

        print(f"✅ Modelo cargado exitosamente")
        print(f"⚡ Tiempo de carga: {load_time:.2f} segundos")
        print(f"📊 Dimensiones: {dimensiones}")

        return True

    except Exception as e:
        print(f"❌ Error cargando {nombre_modelo}: {str(e)}")
        print(f"💡 Probá con otro modelo o verificá tu conexión a internet")
        return False

def evaluar_modelo(button):
    """Evalúa el modelo seleccionado con las oraciones de prueba"""
    with area_resultados:
        clear_output(wait=True)

        nombre_seleccionado = selector_modelo.value

        # Cargamos el modelo si no está cargado o cambió
        if nombre_modelo_actual != nombre_seleccionado:
            if not cargar_modelo(nombre_seleccionado):
                return

        print(f"\n🧪 Evaluando: {nombre_seleccionado}")
        print("=" * 50)

        try:
            # Generamos embeddings
            start_time = time.time()
            embeddings = modelo_actual.encode(oraciones_prueba)
            inference_time = time.time() - start_time

            print(f"⚡ Tiempo de inferencia: {inference_time:.2f} segundos")
            print(f"📊 {len(oraciones_prueba)} oraciones procesadas")
            print(f"🔢 Dimensiones por embedding: {embeddings.shape[1]}")

            # Calculamos similitudes entre pares relacionados
            similitudes_esperadas = [
                (0, 1, "Comida argentina"),  # Comida argentina vs gastronomía Argentina
                (2, 3, "Fútbol/Soccer"),     # Fútbol vs soccer
                (4, 5, "Ayuda matemáticas"), # Ayuda matemáticas vs álgebra
                (6, 7, "Clima lindo"),       # Clima lindo vs día hermoso
            ]

            print("\n🎯 Similitudes entre oraciones relacionadas:")
            matriz_sim = cosine_similarity(embeddings)

            total_similitud = 0
            for i, j, descripcion in similitudes_esperadas:
                sim = matriz_sim[i][j]
                total_similitud += sim
                print(f"• {descripcion}: {sim:.3f}")
                print(f"  '{oraciones_prueba[i]}'")
                print(f"  '{oraciones_prueba[j]}'")
                print()

            # Calculamos una métrica de calidad promedio
            calidad_promedio = total_similitud / len(similitudes_esperadas)
            print(f"📈 Calidad promedio: {calidad_promedio:.3f}")

            # Interpretación de la calidad
            if calidad_promedio > 0.7:
                print("🌟 Excelente calidad para español")
            elif calidad_promedio > 0.5:
                print("👍 Buena calidad para español")
            elif calidad_promedio > 0.3:
                print("⚠️ Calidad moderada para español")
            else:
                print("❌ Calidad baja para español")

        except Exception as e:
            print(f"❌ Error durante la evaluación: {str(e)}")

# Conectamos el botón a la función
boton_evaluar.on_click(evaluar_modelo)

# Mostramos los widgets
print("🎮 Interfaz Interactiva de Comparación de Modelos")
print("" * 50)
display(widgets.VBox([
    widgets.HBox([selector_modelo, boton_evaluar]),
    area_resultados
]))

🎮 Interfaz Interactiva de Comparación de Modelos



### Información Técnica de los Modelos

Acá tenés detalles técnicos sobre cada modelo incluido en la comparación:

#### 🆕 **Google EmbeddingGemma (Nuevo)**
- **Modelo**: `google/embeddinggemma-300m`
- **Parámetros**: 300M
- **Especialidad**: Último modelo de Google, optimizado para similitud semántica
- **Ventajas**: Arquitectura moderna, eficiente en memoria

#### ⚡ **Multilingual MiniLM (Recomendado para empezar)**
- **Modelo**: `paraphrase-multilingual-MiniLM-L12-v2`
- **Parámetros**: ~118M
- **Especialidad**: Balance perfecto velocidad/calidad, excelente con español
- **Ventajas**: Rápido, confiable, bien documentado

#### 🌟 **E5 Multilingual Large**
- **Modelo**: `intfloat/multilingual-e5-large`
- **Parámetros**: ~560M
- **Especialidad**: Estado del arte en calidad multilingüe
- **Ventajas**: Máxima calidad, muy bueno con español

#### 🇪🇸 **DistilRoBERTa Español**
- **Modelo**: `sentence-transformers/paraphrase-spanish-distilroberta`
- **Parámetros**: ~125M
- **Especialidad**: Entrenado específicamente en español
- **Ventajas**: Optimizado para español, entiende jerga regional

#### 🔄 **BGE Multilingual (Estado del Arte)**
- **Modelo**: `BAAI/bge-m3`
- **Parámetros**: ~560M
- **Especialidad**: Último estado del arte en embeddings multilingües
- **Ventajas**: Máximo rendimiento, soporte para 100+ idiomas

### 💡 **Recomendaciones de Uso**

- **Para aprendizaje**: Empezá con Multilingual MiniLM
- **Para producción con calidad**: E5 Multilingual Large o BGE-M3
- **Para español específico**: DistilRoBERTa Español
- **Para experimentar**: Google EmbeddingGemma (muy nuevo)
- **Para velocidad**: Multilingual MiniLM o Universal Sentence Encoder

## Próximos Pasos

Ahora que entendés los conceptos básicos, podés explorar:

1. **Otros modelos**: Probá modelos especializados en español como `sentence-transformers/paraphrase-spanish-distilroberta`
2. **Fine-tuning**: Entrená un modelo específico para tu dominio
3. **Aplicaciones avanzadas**:
   - Sistemas de recomendación
   - Clasificación de textos
   - Traducción automática
   - Chatbots inteligentes

### Modelos Recomendados para Español

```python
# Otros modelos que podés probar:
modelos_español = [
    'sentence-transformers/paraphrase-spanish-distilroberta',
    'sentence-transformers/distiluse-base-multilingual-cased',
    'hiiamsid/sentence_similarity_spanish_es',
]
```

## Resumen

En este cuaderno aprendiste:

✅ **Qué son los embeddings**: Representaciones numéricas del significado de textos

✅ **Cómo generarlos**: Usando modelos pre-entrenados de Hugging Face

✅ **Calcular similitudes**: Con la similitud coseno para medir qué tan parecidos son dos textos

✅ **Diferencias importantes**: Entre embeddings de palabras promediados y embeddings de oraciones contextuales

✅ **Aplicaciones prácticas**: Búsqueda semántica y agrupamiento de textos

✅ **Comparación de modelos**: Herramientas para encontrar el mejor modelo para tus necesidades

Los embeddings son una herramienta fundamental en el procesamiento de lenguaje natural moderno. ¡Seguí experimentando con diferentes textos y modelos para profundizar tu comprensión!